# Pipeline

## Imports

In [1]:
import os
import tensorflow as tf

from preprocessing import preprocess
from dataset import make_dataset
from model import CRNN
from util import *
import random
from datetime import datetime
import wandb
from wandb.keras import WandbCallback

## Paths

In [2]:
DATA_DIR = "../../data/"
TRAIN_DIR = "train-clean-360/LibriSpeech/train-clean-360/"
VALID_DIR = "dev-clean/LibriSpeech/dev-clean/"
TEST_DIR = "test-clean/LibriSpeech/test-clean/"
NOISE_DIR = "noise/"

PROCESSED_DIR = os.path.join(DATA_DIR, "processed/")
DATASET_DIR = os.path.join(DATA_DIR, "dataset/")
MODEL_DIR = "../models/"

## Parameters

#### Dataset Paraameters

In [3]:
initial_s = 10
s = 5
max_k = 10
sample_rate = 16000
preprocessing = False
create_dataset = False
samples = [25, 10, 10] # [1820, 520, 520]
datasets = ["train", "valid", "test"]

#### Training Parameters

In [4]:
lr = 1e-3
epochs = 10
batch_size = 32

## Weights & Biases

In [5]:
wandb.init(project="test-project", entity="speaker-estimation")
wandb.config = {
  "learning_rate": lr,
  "epochs": epochs,
  "batch_size": batch_size
}

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marlousnijman (speaker-estimation). Use `wandb login --relogin` to force relogin


## Preprocessing

In [6]:
if preprocessing:
    input_dirs = [TRAIN_DIR, VALID_DIR, TEST_DIR]

    for i, dataset in enumerate(datasets):
        print(f"Processing {dataset} set")

        input_dir = os.path.join(DATA_DIR, input_dirs[i])
        output_dir = os.path.join(PROCESSED_DIR, f"{dataset}/")

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        preprocess(input_dir, output_dir, initial_s)

## Creating Dataset

In [7]:
if create_dataset:
    for i, dataset in enumerate(datasets):
        print(f"Processing {dataset} set")

        input_dir = os.path.join(PROCESSED_DIR, f"{dataset}/")
        output_dir = os.path.join(DATASET_DIR, f"{dataset}/")

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        make_dataset(input_dir, output_dir, max_k, samples[i])


## Add Noise for k = 0

In [8]:
if create_dataset:
    noise_files = get_files(os.path.join(DATA_DIR, NOISE_DIR, "audio/"), ".wav")

    speech_categories = ["cafe/restaurant", "grocery_store", "metro_station"]
    noise_files = remove_speech_noise(noise_files, os.path.join(DATA_DIR, NOISE_DIR, "meta.txt"), speech_categories)

    for i, n_samples in enumerate(samples):
        print(f"Processing {datasets[i]} set")
        
        # Select same number of noise samples as 
        # k speaker samples in each dataset
        noise_samples = random.sample(noise_files, n_samples)

        # Remove used samples
        [noise_files.remove(n) for n in noise_samples]

        # Downsample and save noise in dataset directories
        process_noise(DATASET_DIR, datasets[i], noise_samples, sample_rate)


## Compute Train Set Mean and Standard Deviation

In [9]:
mean, std = get_train_mean_std(os.path.join(DATASET_DIR, "train/"))
print(f"Dataset mean: {mean}")
print(f"Dataset std: {std}")

100%|██████████| 1100/1100 [02:17<00:00,  7.97it/s]


Dataset mean: 0.0007768943905830383
Dataset std: 0.002090827329084277


## Train

### Data Generators

In [10]:
train_files = [f for f in os.listdir(os.path.join(DATASET_DIR, "train/")) if f.endswith(".wav")]
valid_files = [f for f in os.listdir(os.path.join(DATASET_DIR, "valid/")) if f.endswith(".wav")]

In [11]:
train_generator = CustomDataGenerator(os.path.join(DATASET_DIR, "train/"), 
                                    train_files, dim=(500, 201), 
                                    max_k=max_k, batch_size=batch_size, 
                                    mean=mean, std=std, 
                                    s=s, shuffle=True)

valid_generator = CustomDataGenerator(os.path.join(DATASET_DIR, "valid/"), 
                                    valid_files, dim=(500, 201), 
                                    max_k=max_k, batch_size=1, 
                                    mean=mean, std=std, 
                                    s=s, shuffle=False)

### Model

In [12]:
model = CRNN((500, 201, 1), max_k)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 498, 199, 64)      640       
                                                                 
 conv2d_1 (Conv2D)           (None, 496, 197, 32)      18464     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 165, 65, 32)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 163, 63, 128)      36992     
                                                                 
 conv2d_3 (Conv2D)           (None, 161, 61, 64)       73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 53, 20, 64)       0         
 2D)                                                    

### Compile

In [13]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=tf.keras.metrics.CategoricalAccuracy(),
)

### Callbacks

In [14]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(MODEL_DIR, 'weights.{epoch:02d}-{val_loss:.2f}.h5',),
    save_weights_only=True,
    monitor='val_categorical_accuracy',
    mode='max',
    save_best_only=True)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
)

wandb_callback = WandbCallback(save_model=False)

### Model Directory

In [15]:
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

### Train

In [17]:
model.fit(train_generator, validation_data=valid_generator, epochs=epochs, callbacks=[checkpoint, early_stopping, wandb_callback])
model.save_weights(os.path.join(MODEL_DIR, f'final_weights.h5'))